In [1]:
import os
import sys
import glob
import shutil
import zipfile
import subprocess
import pandas as pd

In [3]:
LANG = "java"
MODEL_TYPE = "roberta"
PRETRAINED_MODEL = "microsoft/codebert-base-mlm"

N_EPOCHS = 15
LEARNING_RATE = 5e-5

BEAM_SIZE = 10
MAX_SOURCE_LEN = 256
MAX_TARGET_LEN = 128

BATCH_SIZE_EVAL = 256
BATCH_SIZE_TRAIN = 256

MINI_DATASET_SIZE = 4096
MINI_MODE_ENABLED = False

FILENAME_JSONL_TEST = "test.jsonl"
FILENAME_JSONL_TRAIN = "train.jsonl"
FILENAME_JSONL_VALID = "valid.jsonl"

FILENAME_JSONL_TEST_MINI = "test-mini.jsonl"
FILENAME_JSONL_TRAIN_MINI = "train-mini.jsonl"
FILENAME_JSONL_VALID_MINI = "valid-mini.jsonl"

FILENAME_CSV_BLEU_SCORES = "bleu_scores.csv"
FILENAME_CSV_EVAL_LOSSES = "eval_losses.csv"
FILENAME_CSV_TRAIN_LOSSES = "train_losses.csv"
FILENAME_TXT_BLEU_SCORE_TEST = "bleu_score.test"

PLT_FIG_SIZE: int = (2 * 2 * 2 * 2, 2 * 2 * 2)

In [5]:
notebook_path = os.path.abspath(os.curdir)
root_path = os.path.dirname(notebook_path)

repo_path = os.path.join(root_path, "repos", "CodeXGLUE")
task_path = os.path.join(repo_path, "Code-Text", "code-to-text")

src_root_path = os.path.join(root_path, "src", "python")
src_task_path = os.path.join(src_root_path, "codexglue", "summarization")

code_path = os.path.join(task_path, "code")
dataset_path = os.path.join(task_path, "dataset")
evaluator_path = os.path.join(task_path, "evaluator")

model_path = os.path.join(notebook_path, "model")
src_notebook_path = os.path.join(notebook_path, "src")

model_name = "{dataset}-{task}-{model_type}-{pretrained_model}-{lang}-n_epochs={n_epochs}-lr={lr}" \
    .format(
        lang=LANG
        , lr=LEARNING_RATE
        , n_epochs=N_EPOCHS
        , dataset="codexglue"
        , task="summarization"
        , model_type=MODEL_TYPE
        , pretrained_model=PRETRAINED_MODEL.replace("/", "_").replace("-", "_")
    )

In [6]:
shutil.copy(
    src=os.path.join(src_root_path, "utils.py")
    , dst=os.path.join(src_notebook_path)
)

shutil.copy(
    src=os.path.join(src_root_path, "iteration_data_point.py")
    , dst=os.path.join(src_notebook_path)
)

sys.path.append(src_notebook_path)

from utils import Utils
from iteration_data_point import IterationDataPoint

In [5]:
if os.path.exists(path=os.path.join(model_path)):
    shutil.rmtree(
        ignore_errors=True
        , path=os.path.join(model_path)
    )

In [6]:
if not os.path.exists(repo_path):
    os.makedirs(repo_path, exist_ok=True)
    
    subprocess.check_call(
        [
            "git"
            , "clone"
            , "https://github.com/microsoft/CodeXGLUE"
            , repo_path
        ]
    )

shutil.copy(
    src=os.path.join(src_root_path, "iteration_data_point.py")
    , dst=os.path.join(code_path)
)

shutil.copy(
    src=os.path.join(src_task_path, "run.py")
    , dst=os.path.join(code_path, "run.py")
)

'/home/user/CS5814-project/repos/CodeXGLUE/Code-Text/code-to-text/code/run.py'

In [7]:
if not os.path.exists(dataset_path):
    with zipfile.ZipFile(os.path.join(task_path, "dataset.zip"), 'r') as dataset_zip_file:
        dataset_zip_file.extractall(task_path)

    for lang in ["python", "java", "ruby", "javascript", "go", "php"]:
        subprocess.check_call(
            [
                "wget"
                , "https://s3.amazonaws.com/code-search-net/CodeSearchNet/v2/{lang}.zip".format(lang=lang)
                , "-P"
                , dataset_path
            ]
        )

        with zipfile.ZipFile(os.path.join(dataset_path, "{lang}.zip".format(lang=lang)), 'r') as lang_zip_file:
            lang_zip_file.extractall(dataset_path)
    
    os.chdir(dataset_path)
    subprocess.check_call(
        [
            "python"
            , os.path.join(dataset_path, "preprocess.py")
        ]
    )
    os.chdir(root_path)

    [os.remove(f) for f in glob.glob(os.path.join(dataset_path, "*.zip"))]
    [os.remove(f) for f in glob.glob(os.path.join(dataset_path, "*.pkl"))]
    [shutil.rmtree(dir, ignore_errors=True) for dir in glob.glob(os.path.join(dataset_path, "*/final"))]

    for lang in ["python", "java"]:
        with open(os.path.join(dataset_path, lang, FILENAME_JSONL_TRAIN), mode="r") as f:
            with open(os.path.join(dataset_path, lang, FILENAME_JSONL_TRAIN_MINI), mode="a") as mini_f:
                for i in range(MINI_DATASET_SIZE):
                    mini_f.write(next(f))

        with open(os.path.join(dataset_path, lang, FILENAME_JSONL_VALID), mode="r") as f:
            with open(os.path.join(dataset_path, lang, FILENAME_JSONL_VALID_MINI), mode="a") as mini_f:
                for i in range(MINI_DATASET_SIZE):
                    mini_f.write(next(f))

        with open(os.path.join(dataset_path, lang, FILENAME_JSONL_TEST), mode="r") as f:
            with open(os.path.join(dataset_path, lang, FILENAME_JSONL_TEST_MINI), mode="a") as mini_f:
                for i in range(MINI_DATASET_SIZE):
                    mini_f.write(next(f))

In [8]:
subprocess.check_call(
    [
        "python"
        
        , os.path.join(code_path, "run.py")
        
        , "--model_type", MODEL_TYPE
        , "--model_name_or_path", PRETRAINED_MODEL
        , "--output_dir", model_path
        
        , "--num_train_epochs", str(N_EPOCHS)
        , "--learning_rate", str(LEARNING_RATE)

        , "--beam_size", str(BEAM_SIZE)
        , "--max_source_length", str(MAX_SOURCE_LEN)
        , "--max_target_length", str(MAX_TARGET_LEN)

        , "--do_train"
        , "--train_batch_size", str(BATCH_SIZE_TRAIN)
        , "--train_filename", os.path.join(
            dataset_path
            , LANG
            , FILENAME_JSONL_TRAIN if not MINI_MODE_ENABLED else FILENAME_JSONL_TRAIN_MINI
        )

        , "--do_eval"
        , "--eval_batch_size", str(BATCH_SIZE_EVAL)
        , "--dev_filename", os.path.join(
            dataset_path
            , LANG
            , FILENAME_JSONL_VALID if not MINI_MODE_ENABLED else FILENAME_JSONL_VALID_MINI
        )

        , "--eval_losses_csv_filename", os.path.join(model_path, FILENAME_CSV_EVAL_LOSSES)
        , "--train_losses_csv_filename", os.path.join(model_path, FILENAME_CSV_TRAIN_LOSSES)
        , "--bleu_scores_csv_filename", os.path.join(model_path, FILENAME_CSV_BLEU_SCORES)
    ]
)

05/07/2022 05:19:26 - INFO - __main__ -   Namespace(adam_epsilon=1e-08, beam_size=10, bleu_score_test_txt_filename=None, bleu_scores_csv_filename='/home/user/CS5814-project/codexglue-summarization-roberta-codebertmlm-java/model/bleu_scores.csv', config_name='', dev_filename='/home/user/CS5814-project/repos/CodeXGLUE/Code-Text/code-to-text/dataset/java/valid.jsonl', do_eval=True, do_lower_case=False, do_test=False, do_train=True, eval_batch_size=256, eval_losses_csv_filename='/home/user/CS5814-project/codexglue-summarization-roberta-codebertmlm-java/model/eval_losses.csv', eval_steps=-1, gradient_accumulation_steps=1, learning_rate=5e-05, load_model_path=None, local_rank=-1, max_grad_norm=1.0, max_source_length=256, max_steps=-1, max_target_length=128, model_name_or_path='microsoft/codebert-base-mlm', model_type='roberta', no_cuda=False, num_train_epochs=15, output_dir='/home/user/CS5814-project/codexglue-summarization-roberta-codebertmlm-java/model', seed=42, test_filename=None, tokeni

0

In [9]:
T = pd.read_csv(os.path.join(model_path, FILENAME_CSV_TRAIN_LOSSES))

Utils.multiline_plot(
    title=model_name
    , fig_size=PLT_FIG_SIZE
    , x_axis_label="Iterations"
    , y_axis_label="Training Loss"
    , yss=[
        T[IterationDataPoint.COL_NAME_METRIC]
    ]
    , xs=T[IterationDataPoint.COL_NAME_ITER_IDX]
    , y_legends=["Training Loss"]
    , xticks_step=len(T) // 20
)

In [10]:
E = pd.read_csv(os.path.join(model_path, FILENAME_CSV_EVAL_LOSSES))
T = pd.read_csv(os.path.join(model_path, FILENAME_CSV_TRAIN_LOSSES))

Utils.multiline_plot(
    title=model_name
    , fig_size=PLT_FIG_SIZE
    , x_axis_label="Epochs"
    , y_axis_label="Loss"
    , yss=[
        T[IterationDataPoint.COL_NAME_METRIC]
        , E[IterationDataPoint.COL_NAME_METRIC]
    ]
    , xs=E[IterationDataPoint.COL_NAME_EPOCH_IDX]
    , y_legends=["Training Loss", "Evaluation Loss"]
)

In [11]:
B = pd.read_csv(os.path.join(model_path, FILENAME_CSV_BLEU_SCORES))

Utils.multiline_plot(
    title=model_name
    , fig_size=PLT_FIG_SIZE
    , x_axis_label="Epochs"
    , y_axis_label="Bleu-4 Score"
    , yss=[
        B[IterationDataPoint.COL_NAME_METRIC]
    ]
    , xs=B[IterationDataPoint.COL_NAME_EPOCH_IDX]
    , y_legends=["Bleu-4 Score"]
)

In [12]:
subprocess.check_call(
    [
        "python"
        
        , os.path.join(code_path, "run.py")
        
        , "--model_type", MODEL_TYPE
        , "--model_name_or_path", PRETRAINED_MODEL
        , "--output_dir", model_path
        , "--load_model_path", os.path.join(model_path, "checkpoint-best-bleu", "pytorch_model.bin")

        , "--beam_size", str(BEAM_SIZE)
        , "--max_source_length", str(MAX_SOURCE_LEN)
        , "--max_target_length", str(MAX_TARGET_LEN)

        , "--do_test"
        , "--eval_batch_size", str(BATCH_SIZE_EVAL)
        , "--test_filename", os.path.join(
            dataset_path
            , LANG
            , FILENAME_JSONL_TEST if not MINI_MODE_ENABLED else FILENAME_JSONL_TEST_MINI
        )

        , "--bleu_score_test_txt_filename", os.path.join(model_path, FILENAME_TXT_BLEU_SCORE_TEST)
    ]
)

05/07/2022 06:52:48 - INFO - __main__ -   Namespace(adam_epsilon=1e-08, beam_size=10, bleu_score_test_txt_filename='/home/user/CS5814-project/codexglue-summarization-roberta-codebertmlm-java/model/bleu_score.test', bleu_scores_csv_filename=None, config_name='', dev_filename=None, do_eval=False, do_lower_case=False, do_test=True, do_train=False, eval_batch_size=256, eval_losses_csv_filename=None, eval_steps=-1, gradient_accumulation_steps=1, learning_rate=5e-05, load_model_path='/home/user/CS5814-project/codexglue-summarization-roberta-codebertmlm-java/model/checkpoint-best-bleu/pytorch_model.bin', local_rank=-1, max_grad_norm=1.0, max_source_length=256, max_steps=-1, max_target_length=128, model_name_or_path='microsoft/codebert-base-mlm', model_type='roberta', no_cuda=False, num_train_epochs=3, output_dir='/home/user/CS5814-project/codexglue-summarization-roberta-codebertmlm-java/model', seed=42, test_filename='/home/user/CS5814-project/repos/CodeXGLUE/Code-Text/code-to-text/dataset/ja

0

In [13]:
with open(os.path.join(model_path, FILENAME_TXT_BLEU_SCORE_TEST), 'r') as bleu_score_test_file:
    print("Test Bleu score: ", bleu_score_test_file.readline())

Test Bleu score:  18.2


In [12]:
print("Top 10 test data ground truth summaries:\n")

with open(os.path.join(model_path, "test_0.gold"), mode="r") as f:
    for _ in range(10):
        print(f.readline())

Top 10 test data ground truth summaries:

0	Makes sure the fast - path emits in order .

1	Wraps an ObservableSource into an Observable if not already an Observable .

2	Returns an Observable that emits the events emitted by source ObservableSource in a sorted order based on a specified comparison function .

3	Child Observers will observe the events of the ConnectableObservable on the specified scheduler .

4	Creates an UnicastProcessor with the given internal buffer capacity hint .

5	Creates an UnicastProcessor with the given internal buffer capacity hint and a callback for the case when the single Subscriber cancels its subscription .

6	Tries to subscribe to a possibly Callable source s mapped Publisher .

7	Maps a scalar value into a Publisher and emits its values .

8	Removes all handlers and resets to default behavior .

9	Wraps a CompletableSource into a Maybe .



In [13]:
print("Top 10 test data predicted summaries using the trained {model_name} model:\n".format(model_name=model_name))

with open(os.path.join(model_path, "test_0.output"), mode="r") as f:
    for _ in range(10):
        print(f.readline())

Top 10 test data predicted summaries using the trained codexglue-summarization-roberta-microsoft_codebert_base_mlm-java-n_epochs=15-lr=5e-05 model:

0	Emit a single element in the queue .

1	Wrap an Observable source .

2	Sorts the elements in sorted order .

3	Observes the given ConnectableObserver on the given scheduler .

4	Creates a new unicast processor instance .

5	Create a new durable processor .

6	Try to check if a subscriber is null .

7	Converts a value to a Streamable .

8	Resets all variables .

9	Creates a new Maybe from a CompletableSource .

